# Assistants API Overview (Python SDK)
https://cookbook.openai.com/examples/assistants_api_overview_python

In [2]:
import sys
print(sys.executable)

/usr/local/opt/python@3.11/bin/python3.11


In [4]:
!pip install --upgrade openai


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [7]:
# ensure the openai version ?= 1.2.3
!pip show openai | grep Version

Version: 1.8.0


In [3]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

## Create a Assistant

In [8]:
from openai import OpenAI

client = OpenAI()

assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-3.5-turbo-1106",
)
show_json(assistant)

{'id': 'asst_26pPYuThSIVv0M0ot9VYjJ75',
 'created_at': 1705665548,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

In [10]:
assistant_id = assistant.id

## Create a Thread

In [12]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_aulMiuLYZnDFnbjA1Id2AC04',
 'created_at': 1705665658,
 'metadata': {},
 'object': 'thread'}

In [13]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)
show_json(message)

{'id': 'msg_F7Au2zLTvnfTnMK9zdhXexEG',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
   'type': 'text'}],
 'created_at': 1705665669,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_aulMiuLYZnDFnbjA1Id2AC04'}

In [14]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_spRk1sZ4k6lvBlYkLl9CBLm6',
 'assistant_id': 'asst_26pPYuThSIVv0M0ot9VYjJ75',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1705665742,
 'expires_at': 1705666342,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_aulMiuLYZnDFnbjA1Id2AC04',
 'tools': []}

In [15]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [17]:
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_spRk1sZ4k6lvBlYkLl9CBLm6',
 'assistant_id': 'asst_26pPYuThSIVv0M0ot9VYjJ75',
 'cancelled_at': None,
 'completed_at': 1705665744,
 'created_at': 1705665742,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1705665742,
 'status': 'completed',
 'thread_id': 'thread_aulMiuLYZnDFnbjA1Id2AC04',
 'tools': []}

In [18]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_Aor1efqhSCHOwcgzNhG2iucr',
   'assistant_id': 'asst_26pPYuThSIVv0M0ot9VYjJ75',
   'content': [{'text': {'annotations': [],
      'value': 'Subtract 11 from both sides to isolate x.'},
     'type': 'text'}],
   'created_at': 1705665743,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_spRk1sZ4k6lvBlYkLl9CBLm6',
   'thread_id': 'thread_aulMiuLYZnDFnbjA1Id2AC04'},
  {'id': 'msg_F7Au2zLTvnfTnMK9zdhXexEG',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
     'type': 'text'}],
   'created_at': 1705665669,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'thread_id': 'thread_aulMiuLYZnDFnbjA1Id2AC04'}],
 'object': 'list',
 'first_id': 'msg_Aor1efqhSCHOwcgzNhG2iucr',
 'last_id': 'msg_F7Au2zLTvnfTnMK9zdhXexEG',
 'has_more': False}

In [19]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Could you explain this to me?"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)

{'data': [{'id': 'msg_S6IyfpiZ2GESpQgCFXqCaJ37',
   'assistant_id': 'asst_26pPYuThSIVv0M0ot9VYjJ75',
   'content': [{'text': {'annotations': [],
      'value': 'Sure! When you subtract 11 from both sides of the equation, you get 3x = 3, and then you can solve for x by dividing both sides by 3.'},
     'type': 'text'}],
   'created_at': 1705665925,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_cqRPxj0ivZU56oR3RiemD6cA',
   'thread_id': 'thread_aulMiuLYZnDFnbjA1Id2AC04'}],
 'object': 'list',
 'first_id': 'msg_S6IyfpiZ2GESpQgCFXqCaJ37',
 'last_id': 'msg_S6IyfpiZ2GESpQgCFXqCaJ37',
 'has_more': False}

## Sample

In [20]:
from openai import OpenAI

MATH_ASSISTANT_ID = assistant.id  # or a hard-coded ID like "asst-..."

client = OpenAI()

def submit_message(assistant_id, thread, user_message):
    client.beta.threads.messages.create(
        thread_id=thread.id, role="user", content=user_message
    )
    return client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant_id,
    )


def get_response(thread):
    return client.beta.threads.messages.list(thread_id=thread.id, order="asc")

In [21]:
def create_thread_and_run(user_input):
    thread = client.beta.threads.create()
    run = submit_message(MATH_ASSISTANT_ID, thread, user_input)
    return thread, run


# Emulating concurrent user requests
thread1, run1 = create_thread_and_run(
    "I need to solve the equation `3x + 11 = 14`. Can you help me?"
)
thread2, run2 = create_thread_and_run("Could you explain linear algebra to me?")
thread3, run3 = create_thread_and_run("I don't like math. What can I do?")

# Now all Runs are executing...

In [22]:
import time

# Pretty printing helper
def pretty_print(messages):
    print("# Messages")
    for m in messages:
        print(f"{m.role}: {m.content[0].text.value}")
    print()


# Waiting in a loop
def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run


# Wait for Run 1
run1 = wait_on_run(run1, thread1)
pretty_print(get_response(thread1))

# Wait for Run 2
run2 = wait_on_run(run2, thread2)
pretty_print(get_response(thread2))

# Wait for Run 3
run3 = wait_on_run(run3, thread3)
pretty_print(get_response(thread3))

# Thank our assistant on Thread 3 :)
run4 = submit_message(MATH_ASSISTANT_ID, thread3, "Thank you!")
run4 = wait_on_run(run4, thread3)
pretty_print(get_response(thread3))

# Messages
user: I need to solve the equation `3x + 11 = 14`. Can you help me?
assistant: Sure. The solution is `x = 1`.

# Messages
user: Could you explain linear algebra to me?
assistant: Linear algebra is the branch of mathematics that deals with vector spaces and linear mappings between those spaces.

# Messages
user: I don't like math. What can I do?
assistant: Start with small, practical applications of math that interest you.

# Messages
user: I don't like math. What can I do?
assistant: Start with small, practical applications of math that interest you.
user: Thank you!
assistant: You're welcome! If you have any more questions, feel free to ask.



## Tools

### Code Interpreter

In [23]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}],
)
show_json(assistant)

{'id': 'asst_26pPYuThSIVv0M0ot9VYjJ75',
 'created_at': 1705665548,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}]}

In [24]:
thread, run = create_thread_and_run(
    "Generate the first 20 fibbonaci numbers with code."
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: Generate the first 20 fibbonaci numbers with code.
assistant: The first 20 Fibonacci numbers are: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181



In [25]:
run_steps = client.beta.threads.runs.steps.list(
    thread_id=thread.id, run_id=run.id, order="asc"
)

In [26]:
for step in run_steps.data:
    step_details = step.step_details
    print(json.dumps(show_json(step_details), indent=4))

{'tool_calls': [{'id': 'call_ZuVLk4uAs3BnuCQqnwIg6TVR',
   'code_interpreter': {'input': '# Function to generate the first n Fibonacci numbers\ndef generate_fibonacci_numbers(n):\n    fibonacci_numbers = [0, 1]\n    for i in range(2, n):\n        next_number = fibonacci_numbers[-1] + fibonacci_numbers[-2]\n        fibonacci_numbers.append(next_number)\n    return fibonacci_numbers\n\n# Generate the first 20 Fibonacci numbers\nfibonacci_20 = generate_fibonacci_numbers(20)\nfibonacci_20',
    'outputs': [{'logs': '[0,\n 1,\n 1,\n 2,\n 3,\n 5,\n 8,\n 13,\n 21,\n 34,\n 55,\n 89,\n 144,\n 233,\n 377,\n 610,\n 987,\n 1597,\n 2584,\n 4181]',
      'type': 'logs'}]},
   'type': 'code_interpreter'}],
 'type': 'tool_calls'}

null


{'message_creation': {'message_id': 'msg_1LfnLzsoWl48gIA7OeUW7lTc'},
 'type': 'message_creation'}

null


### Retrieval

It's for Q&A over document

In [27]:
# Upload the file
file = client.files.create(
    file=open(
        "data/language_models_are_unsupervised_multitask_learners.pdf",
        "rb",
    ),
    purpose="assistants",
)
# Update Assistant
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    tools=[{"type": "code_interpreter"}, {"type": "retrieval"}],
    file_ids=[file.id],
)
show_json(assistant)

{'id': 'asst_26pPYuThSIVv0M0ot9VYjJ75',
 'created_at': 1705665548,
 'description': None,
 'file_ids': ['file-TaAJs3YA1XJeIyTg073RMg8P'],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-3.5-turbo-1106',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'}, {'type': 'retrieval'}]}

In [28]:
thread, run = create_thread_and_run(
    "What are some cool math concepts behind this ML paper pdf? Explain in two sentences."
)
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: What are some cool math concepts behind this ML paper pdf? Explain in two sentences.
assistant: The paper discusses the development of systems capable of learning from natural language directly without the need for interactive communication, speculating that language models with sufficient capacity will be able to infer and perform tasks demonstrated in natural language sequences regardless of their method of procurement【9†source】. Additionally, the paper addresses the use of Byte Pair Encoding (BPE) as a practical middle ground between character and word level language modeling, effectively interpolating between word level inputs for frequent symbol sequences and character level inputs for infrequent symbol sequences, which allows the model to compute the probability of any Unicode string regardless of pre-processing tokenization or vocabulary size【10†source】.



### Functions

In [29]:
def get_mock_response_from_user_multiple_choice():
    return "a"


def get_mock_response_from_user_free_response():
    return "I don't know."


def display_quiz(title, questions):
    print("Quiz:", title)
    print()
    responses = []

    for q in questions:
        print(q["question_text"])
        response = ""

        # If multiple choice, print options
        if q["question_type"] == "MULTIPLE_CHOICE":
            for i, choice in enumerate(q["choices"]):
                print(f"{i}. {choice}")
            response = get_mock_response_from_user_multiple_choice()

        # Otherwise, just get response
        elif q["question_type"] == "FREE_RESPONSE":
            response = get_mock_response_from_user_free_response()

        responses.append(response)
        print()

    return responses

In [30]:
responses = display_quiz(
    "Sample Quiz",
    [
        {"question_text": "What is your name?", "question_type": "FREE_RESPONSE"},
        {
            "question_text": "What is your favorite color?",
            "question_type": "MULTIPLE_CHOICE",
            "choices": ["Red", "Blue", "Green", "Yellow"],
        },
    ],
)
print("Responses:", responses)

Quiz: Sample Quiz

What is your name?

What is your favorite color?
0. Red
1. Blue
2. Green
3. Yellow

Responses: ["I don't know.", 'a']


In [31]:
function_json = {
    "name": "display_quiz",
    "description": "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {"type": "string"},
            "questions": {
                "type": "array",
                "description": "An array of questions, each with a title and potentially options (if multiple choice).",
                "items": {
                    "type": "object",
                    "properties": {
                        "question_text": {"type": "string"},
                        "question_type": {
                            "type": "string",
                            "enum": ["MULTIPLE_CHOICE", "FREE_RESPONSE"],
                        },
                        "choices": {"type": "array", "items": {"type": "string"}},
                    },
                    "required": ["question_text"],
                },
            },
        },
        "required": ["title", "questions"],
    },
}

In [39]:
assistant = client.beta.assistants.update(
    MATH_ASSISTANT_ID,
    model="gpt-4-1106-preview",
    tools=[
        {"type": "code_interpreter"},
        {"type": "retrieval"},
        {"type": "function", "function": function_json},
    ],
)
show_json(assistant)

{'id': 'asst_26pPYuThSIVv0M0ot9VYjJ75',
 'created_at': 1705665548,
 'description': None,
 'file_ids': ['file-TaAJs3YA1XJeIyTg073RMg8P'],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'retrieval'},
  {'function': {'name': 'display_quiz',
    'description': "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': 'An array of questions, each with a title and potentially options (if multiple choice).',
       'items': {'type': 'object',
        'properties': {'question_text': {'type': 'string'},
         'question_type': {'type': 'string',
          'enum': ['MULTIPLE_CHOICE', 'FREE_RESPONSE'

In [40]:
thread, run = create_thread_and_run(
    "Make a quiz with 2 questions: One open ended, one multiple choice. Then, give me feedback for the responses."
)
run = wait_on_run(run, thread)
run.status

'requires_action'

In [41]:
show_json(run)

{'id': 'run_8J8qKLu78N2BlfGoYVtnD42B',
 'assistant_id': 'asst_26pPYuThSIVv0M0ot9VYjJ75',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1705667430,
 'expires_at': 1705668030,
 'failed_at': None,
 'file_ids': ['file-TaAJs3YA1XJeIyTg073RMg8P'],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': {'submit_tool_outputs': {'tool_calls': [{'id': 'call_TynWhLrtAGk3T4rrtw9HiF2P',
     'function': {'arguments': '{\n  "title": "Math Concepts Quick Quiz",\n  "questions": [\n    {\n      "question_text": "Explain in your own words what a logarithm is.",\n      "question_type": "FREE_RESPONSE"\n    },\n    {\n      "question_text": "Which of the following is an irrational number?",\n      "question_type": "MULTIPLE_CHOICE",\n      "choices": ["3.14", "\\frac{5}{7}", "\\frac{22}{7}", "\\frac{1}{3}", "\\frac{\\beta}{ \\beta + 

In [42]:
import json

# Extract single tool call
tool_call = run.required_action.submit_tool_outputs.tool_calls[0]
name = tool_call.function.name
arguments = json.loads(tool_call.function.arguments)

print("Function Name:", name)
print("Function Arguments:")
arguments

Function Name: display_quiz
Function Arguments:


{'title': 'Math Concepts Quick Quiz',
 'questions': [{'question_text': 'Explain in your own words what a logarithm is.',
   'question_type': 'FREE_RESPONSE'},
  {'question_text': 'Which of the following is an irrational number?',
   'question_type': 'MULTIPLE_CHOICE',
   'choices': ['3.14',
    '\x0crac{5}{7}',
    '\x0crac{22}{7}',
    '\x0crac{1}{3}',
    '\x0crac{\x08eta}{ \x08eta + 1}',
    '\x0crac{22}{7}',
    '\x0crac{1}{3}',
    '\x0crac{\x08eta}{ \x08eta + 1}',
    '√2']}]}

In [45]:
responses = display_quiz(arguments["title"], arguments["questions"])

print("Responses:", responses)

Quiz: Math Concepts Quick Quiz

Explain in your own words what a logarithm is.

Which of the following is an irrational number?
0. 3.14
1. rac{5}{7}
2. rac{22}{7}
3. rac{1}{3}
4. raceta}{eta + 1}
5. rac{22}{7}
6. rac{1}{3}
7. raceta}{eta + 1}
8. √2

Responses: ["I don't know.", 'a']


In [46]:
run = client.beta.threads.runs.submit_tool_outputs(
    thread_id=thread.id,
    run_id=run.id,
    tool_outputs=[
        {
            "tool_call_id": tool_call.id,
            "output": json.dumps(responses),
        }
    ],
)
show_json(run)

{'id': 'run_8J8qKLu78N2BlfGoYVtnD42B',
 'assistant_id': 'asst_26pPYuThSIVv0M0ot9VYjJ75',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1705667430,
 'expires_at': 1705668030,
 'failed_at': None,
 'file_ids': ['file-TaAJs3YA1XJeIyTg073RMg8P'],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1705667430,
 'status': 'queued',
 'thread_id': 'thread_QlXalQHEGQF85zlntt5pj86F',
 'tools': [{'type': 'code_interpreter'},
  {'type': 'retrieval'},
  {'function': {'name': 'display_quiz',
    'description': "Displays a quiz to the student, and returns the student's response. A single quiz can have multiple questions.",
    'parameters': {'type': 'object',
     'properties': {'title': {'type': 'string'},
      'questions': {'type': 'array',
       'description': 'An array of questions, each with a title

In [47]:
run = wait_on_run(run, thread)
pretty_print(get_response(thread))

# Messages
user: Make a quiz with 2 questions: One open ended, one multiple choice. Then, give me feedback for the responses.
assistant: Great effort in taking the quiz. Here's some feedback:

1. For the open-ended question about logarithms, you've indicated that you don't know. A logarithm is the exponent to which a fixed number, the base, must be raised to produce a given number. For example, if we have \( \log_b(x) = y \), then by definition, \( b^y = x \).

2. For the multiple-choice question on irrational numbers, you've selected "3.14." While 3.14 is an approximation of the irrational number π (pi), it is by itself a rational number since it can be written as a fraction (314/100). The correct answer is "√2," which cannot be expressed as a fraction of two integers and therefore is an irrational number.

Feel free to ask any questions if you need further clarification on these topics!

